In [71]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_data}sumstat/mono/"
PATH_SUMSTAT_no_corr = f"{PATH_data}sumstat_no_corr"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"


PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'

In [72]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [73]:
effect = 3
chrom = 21
maf_causal_snp = 0.2
sit = "no_strat_causal"

In [74]:
path_genos = PATH_GENO_segmented + f"chrom_{chrom}/unshuffled/"


In [75]:
onlyfiles_input = [f for f in listdir(
    path_genos) if maf_causal_snp + maf_causal_snp/5 > float(f.split('_')[5]) > maf_causal_snp - maf_causal_snp/5]


In [76]:
popped = onlyfiles_input[round(len(onlyfiles_input)/2)]
snp_nr = 0

In [77]:
popped

'19_5_800_19_maf_0.2_0.25.pkl'

In [78]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}")

0

In [79]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}")


0

In [80]:
PATH_mono = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}"


In [81]:
try:
  causal_file = [f for f in listdir(
      PATH_mono) if f.startswith("from_chrom")][0]
  snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")
except Exception as e:
  print(e)
  snps = pd.read_pickle(path_genos+popped)
  snps.to_pickle(f"{PATH_mono}/from_chrom_{chrom}_{popped}")
  snp = snps.iloc[:, snp_nr]
  name = snps.columns[snp_nr]
  snp.to_pickle(f"{PATH_mono}/causal_snp_{name}.pkl")

list index out of range


In [82]:
causal_file = [f for f in listdir(PATH_mono) if f.startswith("from_chrom")][0]


In [83]:
humans = pd.read_pickle(f"{PATH_data}sumstat/humans_no_causal.pkl")


In [84]:
snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")

name = snps.columns[snp_nr]

snp_effect = snps.iloc[:, snp_nr] * effect
#snp_effect = snps.iloc[:, snp_nr] * (effect/float(maf.split('_')[0]))

humans['no_strat_causal'] = humans['no_strat_no_causal']
humans['strat_causal'] = humans['strat_no_causal']

humans['no_strat_causal'] = humans['no_strat_causal'] + snp_effect
humans['strat_causal'] = humans['strat_causal'] + snp_effect

In [85]:
print(Counter(snp_effect).keys())

dict_keys([3, -3, 0])


In [86]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}")


0

In [87]:
path_output = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}/"


In [88]:
pheno = np.array(humans[sit])

In [89]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [90]:
onlyfiles_input = [f for f in listdir(path_genos)]

In [91]:
for file in onlyfiles_input:
    path_file = path_genos + '/' + file
    genos = pd.read_pickle(path_file)    
    X = genos
    y = pheno
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    sumstat = pd.DataFrame(data={
        'snp': list(est2.pvalues.index)[1:],
        'coef': list(est2.params)[1:],
        'std_err': list(est2.bse)[1:],
        't': list(est2.tvalues)[1:],
        'P': list(est2.pvalues[1:]),
        '0.025': list(est2.conf_int()[0])[1:],
        '0.975': list(est2.conf_int()[1])[1:]
    })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0]
                      for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log10(sumstat['P'])
    sumstat.to_pickle(f"{path_output}/{file}")


In [92]:
path_sumstat = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/"


In [93]:
chrom_files = [f for f in listdir(path_sumstat) if f != causal_file]
chrom_files = [f for f in chrom_files if f != f"causal_snp_{name}.pkl"]
chrom_files = [f for f in chrom_files if f != f"GWAS_mono_{sit}.png"]


In [94]:
chrom_files

['21']

In [95]:
sumstat_array = []
for chrom_file in chrom_files:
    sumstat_files = [f for f in listdir(path_sumstat + chrom_file + '/' + sit + "/")]
    # first do it with the first one
    path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_files[0]
    sumstats = pd.read_pickle(path_file)
    for sumstat_file in sumstat_files[1:]:
        path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_file
        temp = pd.read_pickle(path_file)
        sumstats = pd.concat([sumstats,temp])
    sumstat_array.append(sumstats)

sumstats = sumstat_array[0]
for sumstat in sumstat_array[1:]:
    sumstats = pd.concat([sumstats,sumstat]) 

In [96]:
sumstats = sumstats.reset_index(drop=True)


In [97]:
running_pos = 0
cumulative_pos = []
for chrom, group_df in sumstats.groupby('#CHROM'):
    cumulative_pos.append(group_df['POS'] + running_pos)
    running_pos += group_df['POS'].max()

sumstats['cumulative_pos'] = pd.concat(cumulative_pos)
sumstats['SNP number'] = sumstats.index

In [98]:
causal_chrom = causal_file.split("_")[2]

In [99]:
if sit == 'strat_causal':
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with a population dependent phenotype"
elif sit == 'strat_no_causal':
  title = f"No causal snp but with a population dependent phenotype"
elif sit == "no_strat_causal":
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with no population dependent phenotype"
else:
  title = f"No causal snp and no population dependent phenotype"
sit

'no_strat_causal'

In [100]:
title

'One causal snp with AF of 0.2 on chrom 21 with no population dependent phenotype'

In [101]:
sns.relplot(
    data=sumstats,
    x='cumulative_pos',
    y='-logp',
    aspect=2,
    hue='#CHROM',
    palette='dark'
).set(title=title)
plt.savefig(f"{path_sumstat}/GWAS_mono_{sit}.png")
plt.savefig(f"./GWAS_Common.png")


In [102]:
sumstats

,snp,coef,std_err,t,P,0.025,0.975,#CHROM,POS,-logp,cumulative_pos,SNP number
0,21:10892322:A:G_A,5.523734,3.629059,1.522084,0.128208,-1.595087,12.642554,21,10892322,0.892085,10892322,0
1,21:47368531:A:G_G,-4.124976,2.773560,-1.487249,0.137168,-9.565637,1.315685,21,47368531,0.862746,47368531,1
2,21:32892189:A:G_G,-2.509877,2.963794,-0.846846,0.397222,-8.323704,3.303950,21,32892189,0.400967,32892189,2
3,21:15792338:A:C_C,-0.081659,0.527710,-0.154741,0.877047,-1.116823,0.953506,21,15792338,0.056977,15792338,3
4,21:25645657:G:T_T,-1.125316,9.840950,-0.114350,0.908976,-20.429484,18.178852,21,25645657,0.041448,25645657,4
...,...,...,...,...,...,...,...,...,...,...,...,...
239995,21:44811373:G:T_G,0.198773,0.729283,0.272559,0.785224,-1.231594,1.629140,21,44811373,0.105006,44811373,239995
239996,21:27970452:C:T_C,0.872304,1.148703,0.759382,0.447727,-1.380684,3.125293,21,27970452,0.348987,27970452,239996
239997,21:38150472:A:G_A,-0.020570,0.428385,-0.048019,0.961707,-0.860777,0.819636,21,38150472,0.016957,38150472,239997
239998,21:27971241:C:T_T,0.872304,1.148703,0.759382,0.447727,-1.380684,3.125293,21,27971241,0.348987,27971241,239998


In [103]:
sumstats['causality'] = np.ones(sumstats.shape[0])

In [104]:
sumstats['causality'].mask(sumstats['snp'] == name, 0, inplace=True)


In [105]:
#sumstats = sumstats[sumstats['-logp'] > 5].sort_values(by=['-logp'])

In [106]:

"""
sns.relplot(
    data=sumstats,
    x='cumulative_pos',
    y='-logp',
    aspect=2,
    hue='#CHROM',
    style = "causality",
    palette='dark'
).set(title=title)
plt.savefig(f"{path_sumstat}/GWAS_mono_{sit}.png")
plt.savefig(f"./GWAS_Common.png")
"""

'\nsns.relplot(\n    data=sumstats,\n    x=\'cumulative_pos\',\n    y=\'-logp\',\n    aspect=2,\n    hue=\'#CHROM\',\n    style = "causality",\n    palette=\'dark\'\n).set(title=title)\nplt.savefig(f"{path_sumstat}/GWAS_mono_{sit}.png")\nplt.savefig(f"./GWAS_Common.png")\n'